# ROS 2 - Tema IV: Comunicación Síncrona y Creación de Interfaces (Srv)

## 1. Introducción al Concepto

Hasta ahora hemos usado **Topics**, que funcionan como una emisión de radio (asíncrona). Pero en robótica, a menudo necesitamos una respuesta inmediata a una petición concreta. Aquí entran los **Servicios**.

### La Analogía del Restaurante 🍽️
Imagina que estás en un restaurante:
- **El Cliente**: Eres tú. Realizas una **Petición (Request)**: "Quiero una hamburguesa".
- **El Servidor**: Es el camarero/cocina. Procesa tu pedido y te devuelve una **Respuesta (Response)**: La hamburguesa (o un error si no quedan).

A diferencia de los topics, aquí el cliente **se queda esperando** (o gestiona la espera) hasta que recibe su respuesta. Es una comunicación **punto a punto**.

--- 

## 2. ¿Qué son las Interfaces (.srv)?

Para que el Cliente y el Servidor se entiendan, deben hablar el mismo idioma. Ese idioma se define en un archivo de **Interfaz de Servicio (`.srv`)**.

Un archivo `.srv` tiene una estructura fija dividida por tres guiones:
```text
TIPO_DATO nombre_peticion
---
TIPO_DATO nombre_respuesta
```

### Requisitos Previos para Crear Interfaces
Las interfaces suelen crearse en un paquete separado (tipo `rosidlc_default_generators`) para que otros paquetes puedan usarlas. 

> **Tip de Experto:** Nunca olvides añadir `rosidl_default_generators` en tu `package.xml` y `find_package` en el `CMakeLists.txt` de tu paquete de interfaces, de lo contrario, ROS 2 no sabrá cómo convertir ese archivo de texto en código Python/C++.

--- 

## 3. Manos a la obra: Creación del Servicio Personalizado

Imagina que queremos un robot que clasifique piezas por peso. Crearemos un servicio llamado `ClasificarPieza.srv`.

### Pasos en Terminal (Estructura):
1. `cd ~/ros2_ws/src` 
2. `ros2 pkg create --build-type ament_cmake mi_robot_interfaces` (Las interfaces suelen ser C++ por compatibilidad).
3. `mkdir mi_robot_interfaces/srv` 
4. Crear archivo `ClasificarPieza.srv` dentro de esa carpeta.

**Contenido de `ClasificarPieza.srv`:**
```text
float64 peso_gramos
string material
---
bool aceptada
string mensaje_estado
```
---
## 📚 Gestión de Respuestas Asíncronas

En ROS 2, cuando un **Cliente** solicita un servicio, la respuesta no es instantánea. El tiempo que tarda el **Servidor** en procesar la petición (debido a cálculos, latencia de red o hardware) requiere que el Cliente gestione esa espera sin detener el resto de sus procesos. Para ello, utilizamos los objetos `Future` y la función de control del ejecutor `spin_until_future_complete`.



### a. El Concepto de `Future` (El "Ticket de Espera")
Un objeto **`Future`** (futuro) es un marcador de posición para un resultado que aún no se ha recibido. 

* **Promesa de datos**: Al llamar a un servicio de forma asíncrona mediante `call_async()`, ROS 2 devuelve inmediatamente un objeto `Future` en lugar del resultado real.
* **Estado de la petición**: El `Future` permite al nodo verificar en cualquier momento si la respuesta ha llegado, si sigue pendiente o si ha ocurrido un error, sin necesidad de detener la ejecución del código.
* **Analogía**: Es como el avisador vibratorio que te dan en una cadena de comida rápida. No tienes la comida aún (el resultado), pero tienes un objeto que te avisará cuando esté lista mientras puedes seguir haciendo otras cosas.

### b. El método `spin_until_future_complete`
Por defecto, los nodos de ROS 2 necesitan "girar" (`spin`) para procesar eventos, como la llegada de mensajes o respuestas de servicios.

* **Bloqueo controlado**: `spin_until_future_complete(node, future)` es una instrucción que le dice al sistema: "Mantén este nodo activo y procesando datos del sistema, pero no avances a la siguiente línea de mi script de Python hasta que este `Future` específico tenga un resultado".
* **Diferencia con `spin()`**: Mientras que `rclpy.spin(node)` mantiene el nodo vivo indefinidamente, `spin_until_future_complete` solo lo mantiene activo el tiempo necesario para resolver una tarea específica.
* **Importancia en Clientes**: Se utiliza principalmente en scripts de clientes sencillos o tareas de inicialización para asegurar que tenemos los datos necesarios (como una configuración o una confirmación de hardware) antes de proceder con la lógica principal del robot.

### c. Flujo de Trabajo Típico
Para implementar esto en Python (`rclpy`), el flujo estándar es el siguiente:

1.  **Llamada**: Se invoca `self.cli.call_async(request)`, que genera el `future`.
2.  **Espera**: Se utiliza `rclpy.spin_until_future_complete(self, self.future)`.
3.  **Resultado**: Una vez completado, se accede al dato final mediante `self.future.result()`.

---

> ### 💡 Tip
> En aplicaciones robóticas complejas con interfaces gráficas o múltiples sensores, usar `spin_until_future_complete` dentro de un bucle principal puede "congelar" la aplicación. En esos casos, los profesionales prefieren usar **callbacks** (funciones que se ejecutan solas cuando el `future` termina) para mantener la máxima fluidez del sistema.

--- 

## 4. Código Fuente: El Servidor (Python)

Este nodo escuchará peticiones y decidirá si la pieza es válida basándose en un **Parámetro** (reutilizando conceptos del Tema III).

```python
import rclpy
from rclpy.node import Node
# Importamos nuestra interfaz personalizada
from mi_robot_interfaces.srv import ClasificarPieza 

class ClasificadorNode(Node):
    def __init__(self):
        super().__init__('servidor_clasificador')
        
        # Declaramos un umbral de peso como parámetro
        self.declare_parameter('peso_maximo', 500.0)
        
        # Creamos el servicio: (Tipo, Nombre del servicio, Callback)
        self.srv = self.create_service(
            ClasificarPieza, 
            'clasificar_pieza', 
            self.handle_clasificacion
        )
        self.get_logger().info("Servicio de Clasificación listo.")

    def handle_clasificacion(self, request, response):
        limite = self.get_parameter('peso_maximo').value
        
        if request.peso_gramos > limite:
            response.aceptada = False
            response.mensaje_estado = f"Pieza de {request.material} demasiado pesada."
        else:
            response.aceptada = True
            response.mensaje_estado = "Pieza aceptada para montaje."
            
        self.get_logger().info(f"Petición recibida: {request.peso_gramos}g - Respondido: {response.aceptada}")
        return response

def main():
    rclpy.init()
    node = ClasificadorNode()
    rclpy.spin(node)
    rclpy.shutdown()
```

--- 

## 5. Código Fuente: El Cliente (Python)

El cliente debe ser capaz de enviar la información y esperar la respuesta sin "congelar" el robot.

```python
import rclpy
from rclpy.node import Node
from mi_robot_interfaces.srv import ClasificarPieza

class ClienteClasificador(Node):
    def __init__(self):
        super().__init__('cliente_clasificador')
        self.cli = self.create_client(ClasificarPieza, 'clasificar_pieza')
        
        # Esperar a que el servidor esté online
        while not self.cli.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('Servidor no disponible, reintentando...')
        
        self.req = ClasificarPieza.Request()

    def enviar_peticion(self, peso, mat):
        self.req.peso_gramos = peso
        self.req.material = mat
        # Llamada asíncrona
        self.future = self.cli.call_async(self.req)
        # Nota: En un script sencillo usamos spin_until_future_complete
        return self.future

def main():
    rclpy.init()
    cliente = ClienteClasificador()
    future = cliente.enviar_peticion(600.0, "Acero")
    
    rclpy.spin_until_future_complete(cliente, future)
    
    resultado = future.result()
    print(f"RESULTADO: Aceptada={resultado.aceptada} | Nota: {resultado.mensaje_estado}")
    
    cliente.destroy_node()
    rclpy.shutdown()
```

--- 

## 6. Errores Comunes y Tips de Experto

- **Error de Importación:** Si Python dice que no encuentra `mi_robot_interfaces`, es porque no has hecho `source install/setup.bash` después de compilar.
- **El Servidor no responde:** Comprueba con `ros2 service list` si el nombre del servicio coincide exactamente.
- **Cambios en el .srv:** Si modificas el archivo `.srv`, debes borrar la carpeta `build` e `install` de ese paquete y volver a compilar con `colcon build` para asegurar que las cabeceras se regeneren correctamente.

--- 

## 7. Ejercicios de Consolidación

1. **El Robot Matemático**: Crea una interfaz `Operacion.srv` que reciba dos números y un string (ej: "suma", "multiplica") y devuelva el resultado. Implementa el servidor que realice la lógica.
2. **Integración con IA (Teórico/Práctico)**: Imagina un servicio donde el cliente envía una imagen (tipo `sensor_msgs/Image`) y el servidor devuelve las coordenadas de los objetos detectados. ¿Por qué crees que un servicio es mejor que un topic para una detección bajo demanda?
3. **Reto Turtlesim**: Crea un servicio que reciba un nombre de color (string) y llame internamente al servicio `/spawn` de Turtlesim para crear una tortuga en una posición aleatoria si el color es "verde".